In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import polars as pl

df = pl.read_csv("../data/test_results/test_output_GoldenChrysus.csv")

In [10]:
df.head()

submission_id,formatted_context,user_comment_ids
str,str,str
"""3gh4mi""","""subreddit: stocks title: Earli…","""['ctyc5jf', 'ctydlmd']"""
"""3gkkf4""","""subreddit: gifs title: How to …","""['ctz8yu8', 'ctz9bab', 'ctz9fc…"
"""3grdo6""","""subreddit: stocks title: Do yo…","""['cu0r74h', 'cu0z7lg']"""
"""3i5zoq""","""subreddit: stocks title: Is th…","""['cudlv3i', 'cudm4gg', 'cudmfz…"
"""3gish0""","""subreddit: tampa title: Uber?!…","""['ctyu0lk']"""


In [14]:
user_name = "GoldenChrysus"
content = df[3].get_column("formatted_context").item()


In [28]:
from user_embeddings.utils.teacher_prompt import get_teacher_prompt

prompt = get_teacher_prompt(user_name, content)
prompt

<unknown>:1: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1: SyntaxWarning: invalid escape sequence '\ '


'\nYou are an Expert Interaction Analyzer and Label Synthesizer. Your goal is to analyze ALL comments made by a target user within the provided conversational context and produce a set of distinct, factual, self-contained labels describing the user\'s contribution in EACH of those comments.\n\nInput:\n1. target_user_name: The user whose comments are being analyzed.\n2. user_context: A YAML structure representing the conversation thread.\n\nCore Task: For EACH comment by the target_user_name within the user_context:\n    A. Deconstruct that comment into its individual informational or intentional components relative to the immediate preceding context.\n    B. Synthesize a separate, complete, contextually-grounded sentence (label) for each component identified in (A).\n\nProcess to Follow:\n\n1.  Identify Target Comments: Locate all comments authored by GoldenChrysus within the user_context YAML structure.\n2.  Iterate and Analyze Each Target Comment: Process each identified target comme

In [ ]:
from user_embeddings.utils.get_text_completion import get_text_completion
from user_embeddings.utils.get_text_completion import get_openrouter_client

client = get_openrouter_client()

In [29]:

result = get_text_completion(client, "google/gemma-3-27b-it", prompt)
print(result)

## Chain of Thought:

Here's my reasoning for generating the labels for GoldenChrysus's comments. I've broken down the process for each comment, identifying key context, deconstructing the user's contributions, and then synthesizing the labels adhering to the outlined rules (self-containment, contextual framing, precise action verbs, incorporating substance, and appropriate granularity).

**Comment 1:**

*   **Context:** Initial post proposing a Bernie Sanders tax on HFT to fund public college tuition.
*   **Deconstruction:**
    *   Clarification: States the tax isn't limited to HFT, applying to all sales trades.
    *   Further Clarification: Provides links to sources describing the tax, noting their vagueness and potential limitations to larger firms.
    *   Personal Impact: Expresses concern about the tax's impact on their firm, claiming it could eliminate daily profits.
    *   Conditional Qualification: Comments that the effect is dependent on the tax's application target.
    *